In [505]:
import pandas as pd
train= pd.read_csv("data/train.csv",index_col="PassengerId")
test= pd.read_csv("data/test.csv",index_col="PassengerId")
print(train.shape)
print(test.shape)

(891, 11)
(418, 10)


In [48]:
# 열 만지작
class ColsTouch:
    def __init__(self):  #생성자? 겸 내부참조
        pass
    
    def show_info(self, dataframe=test, col="Age"):
        #     @staticmethod
#     def show_info(dataframe,col):
        print("shape:",dataframe.shape)
        print("mean: ",dataframe[col].mean())
        print("count: ",dataframe[col].count())
        print("min: ",dataframe[col].min())
        print("max: ",dataframe[col].max())
        print("")
        
    def colAdd_Id(self, df):
        cnt= df.shape[0]
        df["Id"]= range(0,cnt) #[0,cnt)
        
    def colAdd_Fam(self, df):
        df["FamilySize"]= df["SibSp"]+ df["Parch"]+1
        
    def colAdd_Embarked(self, df):
        '탑승국가 Nation. str'
        df.loc[df["Embarked"]=="C","Nation"]= "FR"
        df.loc[df["Embarked"].isin(["S","Q"]),"Nation"]= "UK"
        
    def colAdd_Fare(self, df):
        'Fare 등급'
        low= (df["Fare"]< 30)
        mid= ((df["Fare"]>= 30) & (df["Fare"]< 100))
        high= (df["Fare"]>= 100)
        
        df.loc[low,"FareGrade"]= "L"
        df.loc[mid,"FareGrade"]= "M"
        df.loc[high,"FareGrade"]= "H"
        
    def colMod_Fare(self, df):
        fare_mean= df["Fare"].mean()
        df.loc[df["Fare"].isnull(),"Fare"]= fare_mean
        
    def colAdd_Age(self, df):
        'Age 세대'
        child= (df["Age"]<11)
        adoles= ((df["Age"]>=11) & (df["Age"]<20))
        adult_y= ((df["Age"]>=20) & (df["Age"]<50))
        adult_o= (df["Age"]>=50)
        
        df.loc[child,"AgeGen"]= "YY"
        df.loc[adoles,"AgeGen"]= "Y"
        df.loc[adult_y,"AgeGen"]= "O"
        df.loc[adult_o,"AgeGen"]= "OO"
        
    def colMod_Age(self, df):
        '''
        NaN값 평균으로 채우기
        parma dataframe name
        '''
        age_mean= df["Age"].mean()
        df.loc[df["Age"].isnull(),"Age"]= age_mean
        
        

In [49]:
# print(len(train.index))
# print(train.shape[0]) #더 빠르데

In [50]:
ct= ColsTouch()

In [509]:
ct.colMod_Age(train)
ct.colMod_Age(test)

ct.colMod_Fare(train)
ct.colMod_Fare(test)

In [52]:
help(ct.colMod_Age.__doc__)

No Python documentation found for 'NaN값 평균으로 채우기\n        parma dataframe name'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



In [510]:
# ct.colAdd_Age(train)
# ct.colAdd_Age(test)

# ct.colAdd_Id(train)
# ct.colAdd_Fam(train)
# ct.colAdd_Fare(train)
# ct.colAdd_Embarked(train)

In [511]:
# cols1= ["Id","FamilySize","Embarked","Nation","Fare","FareGrade"]
# cols2=["Age","AgeGen"]
# print(train[cols2].head(10))
# print(train.head(10))

In [512]:
# pp.show_info()
# pp.show_info(train,"Age")

In [513]:
# cols=["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]
# print(train[cols].head(8))

# PreProcessing

In [514]:
# import pandas as pd
# train= pd.read_csv("data/train.csv",index_col="PassengerId")
# test= pd.read_csv("data/test.csv",index_col="PassengerId")
# print(train.shape)
# print(test.shape)

In [515]:
class PreProcessing:
    def __init__(self):
        pass
    
    @staticmethod
    def showCols():
        print("isnull: ",train[train["Embarked"].isnull()])
        cols1=["Sex","Embarked","Embarked_C","Embarked_S","Embarked_Q"]
        cols2=["Age","Age_yy","Age_y","Age_o","Age_oo","Fare","Fare_low","Fare_mid","Fare_high"]
        print("train: ", train[cols1+cols2])
        print()
        print("test: ", test[cols1+cols2])
#         print((train.loc[[1],["Sex"]]=="male")|(train.loc[[1],["Sex"]]=="female"))
    
    def oneHotSex(self,df):
        #if (train.loc[[1],["Sex"]]=="male")|(train.loc[[1],["Sex"]]=="female"): #한번만 실행하려면??
        df.loc[df["Sex"]=="female","Sex"]= 1
        df.loc[df["Sex"]=="male","Sex"]= 0

    def oneHotEmbarked(self,df):
        cols=["C","S","Q"]
        for col in cols:
            df["Embarked_"+col]= df["Embarked"]== col
#         # 탑승지역으로 본 탑승국가. T/F
#         df["Nation_FR"]= df["Embarked"]=="C"
#         df["Nation_UK"]= df["Embarked"].isin(["S","Q"])
     
    def oneHotAge(self,df):
        # Age 별 나이대 나누기
        df["Age_yy"]= (df["Age"]<11) 
        df["Age_y"]= ((df["Age"]>=11) & (df["Age"]<20))
        df["Age_o"]= ((df["Age"]>=20) & (df["Age"]<50))
        df["Age_oo"]= (df["Age"]>=50)
      
    def oneHotFare(self,df):
        # Fare 값 별 운임등급 나누기
        df["Fare_low"]= (df["Fare"]< 30)
        df["Fare_mid"]= ((df["Fare"]>= 30) & (df["Fare"]< 100))
        df["Fare_high"]= (df["Fare"]>= 100)

In [516]:
class TreeSubmit:
    def __init__(self):
        from sklearn.tree import DecisionTreeClassifier
        predictions= []
        submit= []
    
    def wateringTree(self):
        featureNames= ["Pclass","Sex","Fare","Embarked_C","Embarked_S","Embarked_Q"]
#         featureNames= ["Pclass","Sex","Fare_low","Fare_mid","Fare_high","Embarked_C","Embarked_S","Embarked_Q"]
        label_name= "Survived"
        X_train= train[featureNames]
        y_train= train[label_name]
        
        model= DecisionTreeClassifier(max_depth=5)
        model.fit(X_train,y_train)
        
        X_test= test[featureNames]
        self.predictions= model.predict(X_test)
    
    # @param str
    def toSave(self, filename):
        self.submit= pd.read_csv("data/gender_submission.csv", index_col="PassengerId")
        self.submit["Survived"]= self.predictions
        self.submit.to_csv("data/"+filename+".csv")
        
    @staticmethod
    def showinfo():
        pass


In [517]:
pp= PreProcessing()

In [518]:
ts= TreeSubmit()

In [519]:
# 한 번만 실행 가능
pp.oneHotSex(train) 
pp.oneHotSex(test) 

pp.oneHotEmbarked(train) 
pp.oneHotEmbarked(test)

# pp.oneHotAge(train) 
# pp.oneHotAge(test)

# pp.oneHotFare(train) 
# pp.oneHotFare(test)

In [520]:
ts.wateringTree()

In [521]:
ts.toSave("myty1120")

In [522]:
# PreProcessing.showCols()